In [49]:
import numpy as np
from tqdm import tqdm
import cv2
import pandas as pd
import h5py
import os
# import shutil

# config global variables

In [50]:
debugMode = True
testWorkflow = True
numberData = 25000
numberTest = 12500
batchSize = 32
epochs = 20

filenameInceptionResNetV2 = 'featuresInceptoinResNetV2'
filenameInceptionV3 = 'featuresInceptionV3'
filenameResNet50 = 'featuresResNet50'

# if testWorkflow:
#     numberData = 1000
#     numberTest = 50
#     batchSize = 4
#     epochs = 2

In [51]:
def extract_feature(MODELTYPE, inputSize, convert2RGB, preprocessInput, fileName):
    #read data
    x = np.zeros((numberData, inputSize, inputSize, 3), dtype=np.uint8)
    test = np.zeros((numberTest, inputSize, inputSize, 3), dtype=np.uint8)
    y = np.zeros((numberData, 1), dtype=np.uint8)

    for index in tqdm(range(0, numberData, 2)):
        x[index] = cv2.resize(cv2.imread('train/cat.%d.jpg' %(index/2)), (inputSize, inputSize))
        x[index + 1] = cv2.resize(cv2.imread('train/dog.%d.jpg' %(index/2)), (inputSize, inputSize))
        if convert2RGB:
            x[index]     = x[index][:,:,::-1]
            x[index + 1] = x[index + 1][:,:,::-1]
        y[index + 1] = 1
    
    for index in tqdm(range(numberTest)):
        test[index] = cv2.resize(cv2.imread('test/%d.jpg' %(index+1)), (inputSize, inputSize))
        if convert2RGB:
            test[index] = test[index][:,:,::-1]

    #construct model    
    inputTensor = Input((inputSize, inputSize, 3))
    modelInput = inputTensor
    modelInput = Lambda(preprocessInput)(modelInput)
    baseModel = MODELTYPE(input_tensor=modelInput, weights='imagenet', include_top=False)
    outputFeature = GlobalAveragePooling2D()(baseModel.output)
    model = Model(baseModel.input, outputFeature)
    
    #predict
    featuresTrain = model.predict(x, verbose=debugMode)
    featuresTest = model.predict(test, verbose=debugMode)
    
    #save to file
    if os.path.exists(fileName):
        os.remove(fileName)
        
    with h5py.File(fileName) as h:
        h.create_dataset("train", data=featuresTrain)
        h.create_dataset("test",  data=featuresTest)
        h.create_dataset("label", data=y)
    
#     #for test
#     if testWorkflow:
#         inputTensorTest = Input(featuresTrain.shape[1:])
#         modelInputTest = inputTensorTest
#         modelInputTest = Dropout(0.5)(modelInputTest)
#         modelInputTest = Dense(1, activation='sigmoid')(modelInputTest)
#         modelTest = Model(inputTensorTest, modelInputTest)

#         modelTest.compile(optimizer='adam',
#                       loss='binary_crossentropy',
#                       metrics=['accuracy'])
        
#         modelTest.fit(featuresTrain, y, batch_size=batchSize, epochs=epochs, validation_split=0.2)

## extract features with ResNet50

In [52]:
import keras
from keras.layers import GlobalAveragePooling2D, Dense, Input, Dropout, Lambda
from keras.models import Model

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as piResNet50

extract_feature(ResNet50, 224, False, piResNet50, filenameResNet50)

100%|██████████| 12500/12500 [01:04<00:00, 194.71it/s]


12500/12500 [==============================] - 2286s 183ms/step


## extract features with InceptionV3

In [53]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as piInceptionV3

extract_feature(InceptionV3, 299, True, piInceptionV3, filenameInceptionV3)

100%|██████████| 12500/12500 [01:22<00:00, 151.89it/s]


12500/12500 [==============================] - 21633s 2s/step


## extract features with InceptionResNetV2

In [54]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as piInceptionResNetV2

extract_feature(InceptionResNetV2, 299, True, piInceptionResNetV2, filenameInceptionResNetV2)

100%|██████████| 12500/12500 [01:23<00:00, 148.85it/s]


12500/12500 [==============================] - 4936s 395ms/step


## predict and write submission

In [55]:
def writeSubmission(pred, predNumber, filename):
    submission = pd.read_csv("sample_submission.csv")

    for index in tqdm(range(predNumber)):
        submission.at[index, "label"] = pred[index]

    submission.to_csv(filename, index=None)

In [56]:
from sklearn.utils import shuffle

xTrainMerge = []
xTestMerge = []

index = 0

for featureFilename in [filenameResNet50, filenameInceptionV3, filenameInceptionResNetV2]:
    index += 1
    with h5py.File(featureFilename, 'r') as h:
        xTrain = np.array(h['train'])
        xTrainMerge.append(xTrain)
        
        xTest = np.array(h['test'])
        xTestMerge.append(xTest)
        
        yTrain = np.array(h['label'])

    #     print (yTrain)
        inputTensor = Input(xTrain.shape[1:])
        modelInput = inputTensor
        modelInput = Dropout(0.25)(modelInput)
        modelInput = Dense(1, activation='sigmoid')(modelInput)
        model = Model(inputTensor, modelInput)

        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epochs, validation_split=0.2)
        
        pred = model.predict(xTest, verbose=True)
        pred = pred.clip(min=0.005, max=0.995)

        writeSubmission(pred, numberTest, 'submission%d.csv' %index)
        

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 50s 2ms/step - loss: 0.0736 - acc: 0.9729 - val_loss: 0.0472 - val_acc: 0.9840
Epoch 2/20
20000/20000 [==============================] - 7s 347us/step - loss: 0.0400 - acc: 0.9863 - val_loss: 0.0406 - val_acc: 0.9860
Epoch 3/20
20000/20000 [==============================] - 7s 346us/step - loss: 0.0378 - acc: 0.9856 - val_loss: 0.0420 - val_acc: 0.9840
Epoch 4/20
20000/20000 [==============================] - 7s 344us/step - loss: 0.0319 - acc: 0.9885 - val_loss: 0.0455 - val_acc: 0.9840
Epoch 5/20
20000/20000 [==============================] - 7s 346us/step - loss: 0.0324 - acc: 0.9882 - val_loss: 0.0445 - val_acc: 0.9850
Epoch 6/20
20000/20000 [==============================] - 7s 350us/step - loss: 0.0299 - acc: 0.9892 - val_loss: 0.0460 - val_acc: 0.9844
Epoch 7/20
20000/20000 [==============================] - 7s 343us/step - loss: 0.0281 - acc: 0.9890 - val_loss: 0.0484 - va

100%|██████████| 12500/12500 [00:00<00:00, 47840.51it/s]


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0558 - acc: 0.9859 - val_loss: 0.0220 - val_acc: 0.9924
Epoch 2/20
20000/20000 [==============================] - 7s 349us/step - loss: 0.0186 - acc: 0.9945 - val_loss: 0.0197 - val_acc: 0.9948
Epoch 3/20
20000/20000 [==============================] - 7s 351us/step - loss: 0.0158 - acc: 0.9950 - val_loss: 0.0194 - val_acc: 0.9932
Epoch 4/20
20000/20000 [==============================] - 8s 378us/step - loss: 0.0140 - acc: 0.9954 - val_loss: 0.0199 - val_acc: 0.9936
Epoch 5/20
20000/20000 [==============================] - 8s 387us/step - loss: 0.0132 - acc: 0.9955 - val_loss: 0.0197 - val_acc: 0.9932
Epoch 6/20
20000/20000 [==============================] - 7s 369us/step - loss: 0.0119 - acc: 0.9961 - val_loss: 0.0287 - val_acc: 0.9912
Epoch 7/20
20000/20000 [==============================] - 8s 379us/step - loss: 0.0115 - acc: 0.9959 - val_loss: 0.0208 - va

100%|██████████| 12500/12500 [00:00<00:00, 65350.57it/s]


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 37s 2ms/step - loss: 0.0480 - acc: 0.9906 - val_loss: 0.0220 - val_acc: 0.9944
Epoch 2/20
20000/20000 [==============================] - 7s 341us/step - loss: 0.0171 - acc: 0.9960 - val_loss: 0.0197 - val_acc: 0.9942
Epoch 3/20
20000/20000 [==============================] - 8s 377us/step - loss: 0.0152 - acc: 0.9966 - val_loss: 0.0197 - val_acc: 0.9934
Epoch 4/20
20000/20000 [==============================] - 7s 367us/step - loss: 0.0136 - acc: 0.9965 - val_loss: 0.0206 - val_acc: 0.9952
Epoch 5/20
20000/20000 [==============================] - 7s 334us/step - loss: 0.0119 - acc: 0.9973 - val_loss: 0.0193 - val_acc: 0.9956
Epoch 6/20
20000/20000 [==============================] - 8s 389us/step - loss: 0.0127 - acc: 0.9963 - val_loss: 0.0189 - val_acc: 0.9948
Epoch 7/20
20000/20000 [==============================] - 7s 365us/step - loss: 0.0104 - acc: 0.9976 - val_loss: 0.0190 - va

100%|██████████| 12500/12500 [00:00<00:00, 66123.59it/s]


## merge features

In [57]:
xTrainMerge = np.concatenate(xTrainMerge, axis=1)
xTestMerge = np.concatenate(xTestMerge, axis=1)

xTrainMerge, yTrain = shuffle(xTrainMerge, yTrain)

inputTensor = Input(xTrainMerge.shape[1:])
modelInput = inputTensor
modelInput = Dropout(0.25)(modelInput)
modelInput = Dense(1, activation='sigmoid')(modelInput)
model = Model(inputTensor, modelInput)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(xTrainMerge, yTrain, batch_size=batchSize, epochs=epochs, validation_split=0.2)   

pred = model.predict(xTestMerge, verbose=True)
pred = pred.clip(min=0.005, max=0.995)

writeSubmission(pred, numberTest, 'submissionMergeFeature.csv')

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 37s 2ms/step - loss: 0.0239 - acc: 0.9922 - val_loss: 0.0151 - val_acc: 0.9954
Epoch 2/20
20000/20000 [==============================] - 8s 391us/step - loss: 0.0117 - acc: 0.9964 - val_loss: 0.0185 - val_acc: 0.9946
Epoch 3/20
20000/20000 [==============================] - 8s 404us/step - loss: 0.0084 - acc: 0.9972 - val_loss: 0.0185 - val_acc: 0.9948
Epoch 4/20
20000/20000 [==============================] - 8s 387us/step - loss: 0.0096 - acc: 0.9970 - val_loss: 0.0137 - val_acc: 0.9958
Epoch 5/20
20000/20000 [==============================] - 8s 415us/step - loss: 0.0056 - acc: 0.9987 - val_loss: 0.0186 - val_acc: 0.9946
Epoch 6/20
20000/20000 [==============================] - 9s 458us/step - loss: 0.0055 - acc: 0.9982 - val_loss: 0.0142 - val_acc: 0.9962
Epoch 7/20
20000/20000 [==============================] - 8s 382us/step - loss: 0.0040 - acc: 0.9984 - val_loss: 0.0162 - va

100%|██████████| 12500/12500 [00:00<00:00, 46701.68it/s]


## merge predictions

In [58]:
for element in list(zip(range(numberTest),submission1['label'], submission2['label'], submission3['label'])):
    labels = element[1:]
    submissionMergePredict.at[element[0], "label"] = max(labels) if min(labels) > 0.5 else (min(labels) if max(labels) < 0.5 else np.mean(labels))
    print(submissionMergePredict.at[element[0], "label"], labels)
    
submissionMergePredict.to_csv('submissionMergePredict.csv', index=None)

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.99

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.40051067372163135 (0.10766229033470154, 0.17468009889125824, 0.9191896319389344)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.0

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.990151822566986, 0.9825802445411682, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.027554951608181, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.0049999998882412

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.6328181525071462 (0.9950000047683716, 0.6241792440414429, 0.2792752087116241)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9704629778862, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.00499999988824129

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.992340087890625, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.0049999998882

...

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.025407386943697933, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.3819152629002929 (0.3784866034984589, 0.004999999888241291, 0.7622591853141785)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.6749499291181564 (0.07968778908252716, 0.9950000047683716, 0.9501619935035706)
0.995

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.012496735900640488, 0.006389256101101637)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.21343710335592428 (0.6303113102912903, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.49683470527331036 (0.9547843933105468, 0.4996754825115204, 0.03604423999786377)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0

0.32434374404450256 (0.9630312323570251, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.7644678354263306, 0.9950000047683716, 0.9727502465248108)
0.0

In [61]:
for element in list(zip(range(numberTest),submission1['label'], submission2['label'], submission3['label'])):
    labels = element[1:]
    submissionMergePredict.at[element[0], "label"] =  np.mean(labels)
    print(submissionMergePredict.at[element[0], "label"], labels)
    
submissionMergePredict.to_csv('submissionMeanPredict.csv', index=None)

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.99

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9494570891062418 (0.8583712577819824, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.99

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.00499999

0.0052586339103678865 (0.0057759019546210775, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.0049999998882412

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.008985869586467743 (0.016957608982920647, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.0049999998

...

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9812007149060568 (0.953602135181427, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291,

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9949468572934469 (0.9950000047683716, 0.9950000047683716, 0.9948405623435974)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9202439387639364 (0.7894052863121033, 0.9950000047683716, 0.9763265252113342)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.99500000476837

0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9787289301554362 (0.9461867809295654, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.2618605112656951 (0.2864657342433929, 0.4941157996654511, 0.004999999888241291)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.07933664135634899 (0.2280099242925644, 0.004999999888241291, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.34532117502142984 (0.9950000047683716, 0.0359635204076767, 0.004999999888241291)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.995000004768

0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9765485326449076 (0.9950000047683716, 0.9950000047683716, 0.9396455883979796)
0.9892447590827942 (0.9950000047683716, 0.9777342677116394, 0.9950000047683716)
0.9902210036913554 (0.9950000047683716, 0.9950000047683716, 0.980663001537323)
0.9917090137799581 (0.9950000047683716, 0.9950000047683716, 0.9851270318031312)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.0900566956649224 (0.007175316102802754, 0.004999999888241291, 0.25799477100372314)
0.004999999888241291 (0.004999999888241291, 0.004999999888241291, 0.004999999888241291)
0.9937869906425476 (0.9950000047683716, 0.9913609623908995, 0.9950000047683716)
0.9919181068738302 (0.9887461066246032, 0.9920082092285156, 0.9950000047683716)
0.9950000047683716 (0.9950000047683716, 0.9950000047683716, 0.9950000047683716)
0.004999999888241291